<a href="https://colab.research.google.com/github/Bonorinoa/MapReduce_Python/blob/main/MapReduce_WordCounter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -f hadoop-3.3.2.tar.gz*

In [2]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz

--2022-04-29 12:19:33--  https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638660563 (609M) [application/x-gzip]
Saving to: ‘hadoop-3.3.2.tar.gz’

hadoop-3.3.2.tar.gz 100%[===================>] 609.07M  25.4MB/s    in 24s     

2022-04-29 12:19:58 (24.9 MB/s) - ‘hadoop-3.3.2.tar.gz’ saved [638660563/638660563]



In [3]:
!tar -xzf hadoop-3.3.2.tar.gz

In [4]:
#copy  hadoop file to user/local
!cp -r hadoop-3.3.2/ /usr/local/

In [5]:
#To find the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


To set the java path, go to /usr/local/hadoop-3.3.2/etc/hadoop/hadoop-env.sh then

* Look for the line `export JAVA_HOME=`
* Replace it with `export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64/`
* Remove the comments (the notebook will automatically save the shell script)

In [6]:
# Chek if Hadoop was correctly installed
!/usr/local/hadoop-3.3.2/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in slave mode
hosts filename                   list of hosts to use in slave mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
    

In [7]:
!pip install mrjob

     |████████████████████████████████| 439 kB 5.0 MB/s 


## The goal is to compute the statistical support for the input list of tuples.
### Problem:

Let I = {i1, i2, ..., in} be a set of n items.
Let D = {t1, t2, ..., tm} be a set of m transactions.

We must identify an association of the form Y → X, where X, Y ⊆ I, and X ∩ Y = 0.

In [8]:
# Load Groceries.csv or tinyGroceries.csv
from google.colab import files
import pandas as pd

uploaded = files.upload()

Saving pg26.txt to pg26.txt


In [9]:
%%file wordcounter.py

from mrjob.job import MRJob
import re

WORD_RE = re.compile(r"[\w']+")


class MRWordFreqCount(MRJob):

    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            yield word.lower(), 1

    def combiner(self, word, counts):
        yield word, sum(counts)

    def reducer(self, word, counts):
        yield word, sum(counts)


if __name__ == '__main__':
    MRWordFreqCount.run()

Writing wordcounter.py


## First, let's get the logic of the code to count pairs of objects right with a smaller example

In [10]:
# Change below the name of the Python/MrJob program and the file(s) used by it
# The datafiles can have extensions (e.g. .csv, .txt)  or be without extension.
!python wordcounter.py pg26.txt

Streaming output truncated to the last 5000 lines.
"renewed"	11
"renewing"	1
"renews"	1
"renounce"	3
"renovation"	1
"renown"	7
"renowned"	7
"repaid"	3
"repair"	5
"repaired"	2
"repairing"	3
"repast"	7
"repealed"	1
"repeat"	2
"repeated"	4
"repel"	2
"repelled"	2
"repent"	5
"repentance"	3
"repentant"	1
"repenting"	3
"repents"	1
"repine"	1
"replace"	1
"replacement"	5
"replenished"	2
"replete"	2
"replied"	44
"reply"	4
"report"	3
"reported"	2
"reports"	1
"repose"	6
"reposed"	2
"repossess"	1
"represent"	2
"representations"	1
"represented"	1
"representing"	2
"reproach"	4
"reproachful"	1
"reprobate"	1
"reproved"	1
"reptile"	1
"repulse"	3
"repulsed"	3
"repute"	2
"request"	9
"require"	5
"required"	2
"requirements"	4
"requires"	4
"rescued"	2
"research"	2
"resemblance"	3
"resemblances"	1
"resembles"	2
"resemblest"	1
"resembling"	2
"resent"	1
"reserve"	1
"reserved"	7
"reserving"	1
"reside"	3
"residence"	3
"resides"	1
"residing"	2
"resign"	5
"resigns"	2
"resist"	6
"resistance"	1
"resistless"	1
"resolu

## Let's run it in Hadoop now

In [11]:
%%file ~/.mrjob.conf
runners:
    hadoop:
      hadoop_bin: /usr/local/hadoop-3.3.2/bin/hadoop
      hadoop_streaming_jar: /usr/local/hadoop-3.3.2/share/hadoop/tools/lib/hadoop-streaming-3.3.2.jar
      hadoop_tmp_dir: file:///content/tmp/mrjob

Writing /root/.mrjob.conf


In [13]:
# Change below the name of the Python/MrJob program and the file(s) used by it
# Note that the option -r hadoop changes the execution  from local to the hadoop framework
!python wordcounter.py -r hadoop  pg26.txt

Streaming output truncated to the last 5000 lines.
"inspires"	2
"inspiring"	1
"instant"	3
"instantly"	1
"instead"	10
"instilled"	2
"instinct"	4
"instinctive"	1
"instruct"	2
"instructed"	2
"instructer"	2
"instruction"	1
"instructs"	1
"instrument"	3
"instrumental"	2
"instruments"	1
"insufferably"	1
"insulting"	2
"insuperable"	1
"insupportable"	1
"insurrection"	1
"integrity"	2
"intellect"	1
"intellectual"	6
"intelligence"	1
"intelligent"	1
"intelligential"	2
"intemperance"	1
"intend"	4
"intended"	6
"intends"	7
"intense"	1
"intent"	9
"intercede"	1
"intercept"	4
"intercession"	1
"intercessour"	3
"interchange"	1
"intercourse"	4
"interdicted"	2
"interdiction"	1
"interested"	1
"interfused"	1
"intermission"	2
"intermit"	1
"intermits"	1
"intermitted"	2
"intermix"	2
"intermixed"	2
"internal"	3
"international"	1
"interpose"	2
"interposed"	5
"interposes"	1
"interposest"	1
"interpret"	1
"interpreted"	2
"interpreter"	2
"interrupt"	5
"interrupted"	1
"interval"	1
"intervene"	1
"interview"	2
"intervolve